In [1]:
from genparse.fst import FST, EPSILON
from genparse import Float
from arsenal import iterview
from collections import Counter

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
print(f'token vocabulary size: {tokenizer.vocab_size}')
T = [(token_id, tokenizer.decode([token_id])) for token_id in range(tokenizer.vocab_size)]

token vocabulary size: 50257


In [3]:
import random
random.seed(8675309)
S = random.sample(T, 2000)
#S = T
S

[(26361, 'zon'),
 (14032, ' purple'),
 (33439, ' MIDI'),
 (43499, 'announced'),
 (33034, ' hardened'),
 (18411, ' thermal'),
 (42034, 'devices'),
 (42415, ' ankles'),
 (44356, ' Turtles'),
 (14886, ' Tyler'),
 (42111, ' infiltrate'),
 (46641, ' vividly'),
 (22999, ' burnt'),
 (5973, 'icken'),
 (817, 'Th'),
 (32498, ' clones'),
 (42535, '}}}'),
 (20933, ' warehouse'),
 (31641, ' Rw'),
 (3200, ' secret'),
 (15114, ' clouds'),
 (24081, ' Pav'),
 (40220, '1973'),
 (45577, ' Lauderdale'),
 (22846, 'ANN'),
 (9409, 'isd'),
 (12374, 'ettings'),
 (18180, ' rivers'),
 (25353, ' Background'),
 (14977, ' arbitrary'),
 (15545, ' Install'),
 (48465, ' microbiome'),
 (35094, ' Gau'),
 (40935, ' Daesh'),
 (17410, 'Method'),
 (41589, 'Harris'),
 (8975, ' Sometimes'),
 (29173, '644'),
 (12187, ' cake'),
 (34909, ' disabling'),
 (4849, ' Sal'),
 (22507, ' Allow'),
 (34480, ' Colony'),
 (2439, ' Sm'),
 (709, 'ov'),
 (47261, 'wat'),
 (16069, ' ranges'),
 (13431, ' frames'),
 (39911, ' Lov'),
 (35905, ' inf

In [4]:
def bpe_wfst(S):
    m = FST(Float)
    START = 0
    STOP = 1
    m.add_I(0, 1)
    for i, x in iterview(S): 
        m.add_arc(START, (i, EPSILON), (i, 0), 1)
        for j in range(len(x)):
            m.add_arc((i,j), (EPSILON, x[j]), (i,j+1), 1)
        m.add_arc((i,len(x)), (EPSILON, EPSILON), STOP, 1)
    m.add_F(STOP, 1)
    m.add_arc(STOP, (EPSILON, EPSILON), START, .1)   # decay
    return m.renumber

In [5]:
T_b_c = bpe_wfst(S)

Output()

In [6]:
#pl.hist([len(B) for B in iterview(list(E.blocks()))])

In [7]:
e = T_b_c(None, 'zon purple MIDI Turtlesicken').trim

In [8]:
len(e.trim_vals.states)

43

In [20]:
len(e.states)

43

In [21]:
e.epsremove.trim

In [22]:
len(e.trim.states)

43

In [23]:
#e = T_b_c.project(1).trim

In [24]:
#e('zon purple MIDI Turtlesicken')

In [25]:
from genparse.util import LarkStuff

In [26]:
lark_stuff = LarkStuff("""

start: [ KEYWORD | INTEGER | BOOLEAN ] EOS

EOS: "</s>"
NAME: /[A-Za-z]+/
KEYWORD: "SELECT" | "WHERE" | "FROM"
INTEGER: /[1-9][0-9]*/
BOOLEAN: "true"  | "false"
    
""")

In [27]:
m = lark_stuff.transducer()

In [28]:
tmp = m('SELECT', None).trim.epsremove.trim
tmp

In [29]:
tmp = m('SELECT\n\t\r FROM 1 2', None).trim.epsremove.trim#.min
tmp

In [30]:
tmp = m('SELECT\nFROM', ('KEYWORD', 'KEYWORD'))
tmp

0.99